In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from glob import glob
import itertools
from pao_file_utils import parse_pao_file

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [ ]:
from spherical_harmonics import Y_l, get_clebsch_gordan_coefficients_sympy, get_clebsch_gordan_coefficients_table #, get_clebsch_gordan_coefficients_new
get_clebsch_gordan_coefficients = get_clebsch_gordan_coefficients_sympy
#get_clebsch_gordan_coefficients = get_clebsch_gordan_coefficients_table
#get_clebsch_gordan_coefficients = get_clebsch_gordan_coefficients_new


In [ ]:
def convolute(coords, atom2kind, central_atom, max_l):
    natoms = coords.shape[0]
    assert coords.shape[1] == 3
    kind_names = list(sorted(set(atom2kind)))
    integrals = []
    for l in range(max_l + 1):
        for sigma in [0.5, 1.0, 2.0, 3.0, 4.0]:
            for ikind in kind_names:
                integrals.append(np.zeros(2*l + 1))
                for iatom in range(natoms):
                    if atom2kind[iatom] == ikind and iatom != central_atom:
                        r = coords[central_atom] - coords[iatom]
                        angular_part = Y_l(r, l)
                        radial_part = np.exp(- np.dot(r,r) / sigma**2)
                        integrals[-1] += radial_part * angular_part
    return integrals

In [ ]:
def combinations(channels, max_l):
    """ returns all possbile combinations of input channels up to given max_l """
    output_channels = list()
    for channel_i, channel_j in itertools.combinations_with_replacement(channels, 2):
        assert len(channel_i.shape) == len(channel_j.shape) == 1
        li = (channel_i.size - 1) // 2
        lj = (channel_j.size - 1) // 2
        # There li + lj possible ways to combine the two channels.
        # We do all of them up to a max_l.
        lo_min = abs(li-lj)
        lo_max = min(li+lj, max_l)
        for lo in range(lo_min, lo_max + 1): # l of output
            channel_o = np.zeros(2*lo+1)
            cg = get_clebsch_gordan_coefficients(li, lj, lo)
            channel_o = np.einsum("i,j,ijo->o", channel_i, channel_j, cg)
            output_channels.append(channel_o)
    return output_channels

In [ ]:
# Load training data and hard code metadata.
#pao_files = sorted(glob("2H2O_MD/frame_*/2H2O_pao44-1_0.pao"))
#pao_files = sorted(glob("2H2O_rotations/phi_*/2H2O_pao44-1_0.pao"))
pao_files = sorted(glob("2H2O_rotations/*/2H2O_pao44-1_0.pao"))

prim_basis_shells = {
    'H': [2, 1, 0], # two s-shells, one p-shell, no d-shells
    'O': [2, 2, 1], # two s-shells, two p-shells, one d-shell
}

pao_basis_size = 4

In [ ]:
class Sample:
    def __init__(self, channels, xblock, iatom, coords):
        self.channels = channels
        self.xblock = xblock
        self.iatom = iatom # just for debugging
        self.coords = coords # just for debugging
        #TODO generalize
        self.s_channel_list = [c for c in channels if c.size == 1]
        self.p_channel_list = [c for c in channels if c.size == 3]
        self.d_channel_list = [c for c in channels if c.size == 5]

        self.s_channels = np.concatenate(self.s_channel_list)
        self.p_channels = np.stack(self.p_channel_list).T
        if (self.d_channel_list):
            self.d_channels = np.stack(self.d_channel_list).T
        else:
            self.d_channels = np.zeros(shape=[])

def build_dataset(kind_name, max_l):
    samples = []
    for fn in pao_files:
        kinds, atom2kind, coords, xblocks = parse_pao_file(fn)
        natoms = coords.shape[0]
        for iatom in range(natoms):
            if atom2kind[iatom] == kind_name:
                initial_channels = convolute(coords, atom2kind, iatom, max_l)
                if True: # apply combination
                    comb_channels = combinations(initial_channels, max_l)
                    sample = Sample(comb_channels, xblocks[iatom], iatom, coords)
                else:
                    sample = Sample(initial_channels, xblocks[iatom], iatom, coords)
                samples.append(sample)

    print("\nDataset for: ", kind_name)
    print("samples: ", len(samples))
    print("s channels: ", len(samples[0].s_channel_list))
    print("p channels: ", len(samples[0].p_channel_list))
    print("d channels: ", len(samples[0].d_channel_list))
    return samples
    
def samples2xy(samples):
    s_input = np.array([s.s_channels for s in samples], np.float32)
    p_input = np.array([s.p_channels for s in samples], np.float32)
    d_input = np.array([s.d_channels for s in samples], np.float32)
    x_data = [s_input, p_input, d_input]
    y_data = np.array([s.xblock for s in samples], np.float32)
    return x_data, y_data

H_dataset = build_dataset("H", max_l=1)
O_dataset = build_dataset("O", max_l=2)

In [ ]:
#TODO implement custom layer instead of Dense + Lambda:
def contract(args):
    assert len(args) == 2
    coeffs = args[0]
    channels = args[1]
    return tf.reduce_sum(coeffs[...,None,:] * channels, axis=-1)

def build_model(first_sample, pao_basis_size, prim_basis_shells, num_hidden_layers, hidden_layers_width):
    
    prim_basis_size = sum([(2*l+1)*n for l, n in enumerate(prim_basis_shells)])
    num_p_channels = len(first_sample.p_channel_list)
    num_d_channels = len(first_sample.d_channel_list)
    
    # define two sets of inputs
    s_input = layers.Input(shape=first_sample.s_channels.shape, name="s_input")
    p_input = layers.Input(shape=first_sample.p_channels.shape, name="p_input")
    d_input = layers.Input(shape=first_sample.d_channels.shape, name="d_input")
    
    #TODO: add more hidden layers
    hidden_out = s_input
    for i in range(num_hidden_layers):
        hidden_out = layers.Dense(hidden_layers_width, activation='softmax', name=f"hidden_layer_{i}")(hidden_out)
            
    pao_output = []
    for i in range(pao_basis_size):
        for l, n in enumerate(prim_basis_shells):
            for j in range(n):
                name = f"{i}{l}{j}"
                if l == 0:
                    s_out = layers.Dense(1, name="coeffs_"+name)(hidden_out)
                    pao_output.append(s_out)
                elif l == 1:
                    p_coeffs = layers.Dense(num_p_channels, name="coeffs_"+name)(hidden_out)
                    contract_layer = layers.Lambda(contract, name="contr_"+name)
                    p_out = contract_layer([p_coeffs, p_input])
                    pao_output.append(p_out)
                elif l == 2:
                    d_coeffs = layers.Dense(num_d_channels, name="coeffs_"+name)(hidden_out)
                    contract_layer = layers.Lambda(contract, name="contr_"+name)
                    d_out = contract_layer([d_coeffs, d_input])
                    pao_output.append(d_out)
                else:
                    raise Exception("Not implemented")
    
    xvec = layers.concatenate(pao_output)
    xblock = layers.Reshape((pao_basis_size, prim_basis_size))(xvec) #TODO: maybe transpose?

    inputs = [s_input, p_input, d_input]
    model = keras.Model(inputs=inputs, outputs=xblock)
    #model.summary()
    return(model)

H_model = build_model(H_dataset[0], pao_basis_size, prim_basis_shells['H'], num_hidden_layers=10, hidden_layers_width=10)
O_model = build_model(O_dataset[0], pao_basis_size, prim_basis_shells['O'], num_hidden_layers=10, hidden_layers_width=10)

In [ ]:
def loss_function(xblock_true, xblock_pred):
    #TODO: This might not be ideal as it implicitly forces the predicted basis vectors to be orthonormal.
    projector = tf.matmul(xblock_pred, xblock_pred, transpose_a=True)
    residual = xblock_true - tf.matmul(xblock_true, projector)
    return tf.reduce_mean(tf.pow(residual, 2))

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
H_model.compile(optimizer, loss=loss_function)
O_model.compile(optimizer, loss=loss_function)

In [ ]:
class TrainingLogger(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.epoch_counter = 0
    def on_epoch_end(self, epoch, logs={}):
        self.epoch_counter += 1
        if self.epoch_counter % 50 == 0:
            print("Epoch: {} Loss: {}".format(self.epoch_counter, logs['loss']))

In [ ]:
H1_dataset = [s for s in H_dataset if s.iatom==1]
x_train, y_train = samples2xy(H1_dataset[0:1])
H_model.fit(x_train, y_train, epochs=1000, verbose=0, callbacks=[TrainingLogger()]);

losses = []
for s in H1_dataset:
    x_test, y_test = samples2xy([s])
    loss = H_model.evaluate(x_test, y_test, verbose=0)
    losses.append(loss)

plt.yscale('log')
plt.plot(losses);

In [ ]:
O3_dataset = [s for s in O_dataset if s.iatom==3]
x_train, y_train = samples2xy(O3_dataset[0:1])
O_model.fit(x_train, y_train, epochs=1000, verbose=0, callbacks=[TrainingLogger()]);

losses = []
for s in O3_dataset:
    x_test, y_test = samples2xy([s])
    loss = O_model.evaluate(x_test, y_test, verbose=0)
    losses.append(loss)

plt.yscale('log')
plt.plot(losses);

In [ ]:
#x_test, y_test = samples2xy(O3_dataset[1:2])
#print(O_model.predict(x_test)[0,0,:])
print(O_model.predict(x_train)[0,0,:])
#print(y_test[0,0,:])
print(y_train[0,0,:])

In [ ]:
len(H_dataset)

In [ ]:
get_clebsch_gordan_coefficients_new(1,1,1)

In [ ]:
from sympy.physics.quantum.cg import CG, Wigner3j

In [ ]:
CG(1, 0, 1, 1, 1, 2).doit() 

In [ ]:
Wigner3j(1, 0, 1, 0, 1, 0).doit() 

In [ ]:
get_clebsch_gordan_coefficients_sympy(1,1,0)[:,:,0]

In [ ]:
get_clebsch_gordan_coefficients_table(1,1,0)[:,:,0]

In [ ]:
O_dataset[0].d_channels

In [ ]:
j1 = 0.5
j2 = 0.5
j3 = 0
m1 = -0.5
m2 = 0.5
m3 = 0.0
CG(j1, m1, j2, m2, j3, m3).doit() 

In [ ]:
import numpy as np

In [ ]:
def get_clebsch_gordan_coefficients_new(li, lj, lo):
    from sympy.physics.quantum.cg import Wigner3j
    #TODO maybe rename lo -> lk ?
    assert abs(li-lj) <= lo <= abs(li+lj)

    #cg0 = CG(li, 0, lj, 0, lo, 0).doit() 
    #numerator = (2*li+1) * (2*lj+1)
    #denominator = 4 * np.pi * (2*lo+1)
    #pre_factor = cg0 * np.sqrt(numerator 

    coeffs = np.zeros(shape=(2*li+1, 2*lj+1, 2*lo+1))
    for mi in range(-li, li+1):
        for mj in range(-lj, lj+1):
            for mo in range(-lo, lo+1):
                cg = Wigner3j(li, mi, lj, mj, lo, mo).doit()
                coeffs[mi+li, mj+lj, mo+lo]
    return coeffs

get_clebsch_gordan_coefficients_new(1,1,0)[:,:,0]

In [ ]:
def get_clebsch_gordan_coefficients_sympy(li, lj, lo):
    from sympy.physics.quantum.cg import CG
    #TODO maybe rename lo -> lk ?
    assert abs(li-lj) <= lo <= abs(li+lj)
    coeffs = np.zeros(shape=(2*li+1, 2*lj+1, 2*lo+1))
    for mi in range(-li, li+1):
        for mj in range(-lj, lj+1):
            for mo in range(-lo, lo+1):  
                # https://docs.sympy.org/latest/modules/physics/quantum/cg.html
                cg = CG(li, mi, lj, mj, lo, mo).doit()
                #cg = CG(lo, mo, li, mi, lj, mj).doit()
                #factor = pow(-1, mj+1)
                factor = 1.0
                coeffs[mi+li, mj+lj, mo+lo] = cg * factor
    return coeffs

get_clebsch_gordan_coefficients_sympy(1,1,0)[:,:,0]

In [ ]:
get_clebsch_gordan_coefficients_sympy(1,1,1)[:,:,0]

In [ ]:
get_clebsch_gordan_coefficients_table(1,1,1)[:,:,0]

In [ ]:
get_clebsch_gordan_coefficients_table(1,1,1)[:,:,1]

In [ ]:
get_clebsch_gordan_coefficients_table(1,1,1)[:,:,2]

In [ ]:
from sympy.physics.quantum.spin import WignerD
from sympy import pi

In [ ]:
j=1
m=0
mp=0
alpha=pi/10
beta=0
gamma=0
for m1
WignerD(j, m, mp, alpha, beta, gamma).doit()

In [ ]:
def get_wignerD_matrix(l, alpha=0, beta=0, gamma=0):
    D = np.zeros(shape=(2*l+1, 2*l+1), dtype=complex)
    for mi in range(-l, l+1):
        for mo in range(-l, l+1):
            D[mi+l, mo+l] = WignerD(j, mi, mo, alpha, beta, gamma).doit()
    return D

get_wignerD_matrix(l=1, alpha=1.3)

In [ ]:
cg_matrix = get_clebsch_gordan_coefficients_sympy(1,1,1)

In [ ]:
#lie_learn/representations/SO3/clebsch_gordan_numerical.
from lie_learn.representations.SO3 import clebsch_gordan_numerical

In [ ]:
import quadpy

In [ ]:
scheme = quadpy.sphere.lebedev_041()
#scheme = quadpy.sphere.lebedev_003a()

In [ ]:
scheme.points

In [ ]:
scheme.weights.shape

In [ ]:
from spherical_harmonics import Y_l
import numpy as np

In [ ]:
scheme.points

In [ ]:
x = np.sum(Y_l(scheme.points, l=2) * scheme.weights[:,None], axis=0)

In [ ]:
def get_clebsch_gordan_coefficients_num(li, lj, lo):
    assert abs(li-lj) <= lo <= abs(li+lj)
    coeffs = np.zeros(shape=(2*li+1, 2*lj+1, 2*lo+1))
    
    #scheme = quadpy.sphere.lebedev_041()
    scheme = quadpy.sphere.lebedev_011()

    y_i = Y_l(scheme.points, l=li)
    y_j = Y_l(scheme.points, l=lj)
    y_o = Y_l(scheme.points, l=lo)
    y_ijo = y_i[:,:,None,None] * y_j[:,None,:,None] * y_o[:,None,None,:]
    coeffs = np.sum(y_ijo * scheme.weights[:,None,None,None], axis=0)
    return coeffs

get_clebsch_gordan_coefficients_num(1,1,1)

In [ ]:
from spherical_harmonics import get_clebsch_gordan_coefficients_table
get_clebsch_gordan_coefficients_table(1,1,1)

In [ ]:
from spherical_harmonics import get_clebsch_gordan_coefficients_sympy
get_clebsch_gordan_coefficients_sympy(1,1,1)

In [ ]:
from spherical_harmonics import get_clebsch_gordan_coefficients_sympy
get_clebsch_gordan_coefficients_sympy(1,1,1)

In [ ]:
import numpy as np
from sympy.physics.quantum.cg import CG

def foo(li, lj, lo):
    assert abs(li-lj) <= lo <= abs(li+lj)
    coeffs = np.zeros(shape=(2*li+1, 2*lj+1, 2*lo+1))
    for mi in range(-li, li+1):
        for mj in range(-lj, lj+1):
            for mo in range(-lo, lo+1):
                # https://docs.sympy.org/latest/modules/physics/quantum/cg.html
                cg = CG(li, mi, lj, mj, lo, mo).doit()
                coeffs[mi+li, mj+lj, mo+lo] = cg
    return coeffs

foo(1,1,0)


In [ ]:
from scipy.spatial.transform import Rotation

In [ ]:
from spherical_harmonics  import Y_l

In [ ]:
def rot_mat(g):
    # g = [alpha, beta, gamma]
    return  Rotation.from_euler('zyx', g).as_dcm()

def clebsch_gordon_num(l1, l2, l3):
    N = 100
    # pick random directions
    r = np.random.rand(N, 3)
    #assert abs(m1) <= l1
    #assert abs(m3) <= l2
    #assert abs(m3) <= l3
    
    n1 = 2 * l1 + 1
    n2 = 2 * l2 + 1
    A = np.zeros(shape=[N, n1 * n2])
    
    y1 = Y_l(r, l=l1)
    y2 = Y_l(r, l=l2)
    for i1 in range(n1):
        for i2 in range(n2):
            A[:, i1*n2 + i2] =  y1[:,i1] * y2[:,i2]

    b = Y_l(r, l=l3)
    
    
    #print(b.shape)
    A_inv = np.linalg.pinv(A)
    print(A_inv.shape)
    #print(np.dot(A_inv, b).shape)
    print(np.linalg.pinv(A).dot(b))
    
    
    #y1 = Y_l(r, l=l1)[:, l1+m1]
    #y2 = Y_l(r, l=l2)[:, l2+m2]
    #y3 = Y_l(r, l=l3)[:, l3+m3]
    
    
    #print((y1 + y2) / y3)
    #print((c1 * y1 + c2 * y2) / y3)
    
#     for i in range(10):
#         g = np.random.rand(3) * 2 * np.pi
#         R = rot_mat(g)
#         r1R = np.einsum("ki,ij->kj", r1, R)
#         r2R = np.einsum("ki,ij->kj", r2, R)

#         y1R = Y_l(r1R, l=l1)[:, m1-l1]
#         y2R = Y_l(r1R, l=l2)[:, m2-l2]
#         y3R = Y_l(r2R, l=l3)[:, m3-l3]

#         print(y1 * y1 * y3, y1R * y2R * y3R)
    
    #print(np.sum(r1 * r2, axis=1) - np.sum(r1R * r2R, axis=1))

clebsch_gordon_num(l1=1, l2=1, l3=2)
    

In [3]:
import numpy as np
from spherical_harmonics  import Y_l
from scipy.spatial.transform import Rotation

def rot_mat(g):
    # g = [alpha, beta, gamma]
    return  Rotation.from_euler('zyx', g).as_dcm()

def wigner_d_num(g, l):
    """Determine Wigner D-matrix numerically."""
    # g: euler angles
    # pick random directions
    N = 2*2*(l+1)  # use twice as many too be sure
    r1 = np.random.rand(N, 3)
    
    # build rotation matrix
    R = rot_mat(g)
    
    # rotate random direction
    r2 = np.einsum("ki,ij->kj", r1, R)
    
    y1 = Y_l(r1, l=l)
    y2 = Y_l(r2, l=l)
    
    return np.linalg.pinv(y1).dot(y2)

def clebsch_gordan_num(l1, l2, l3):
    """
    \sum_{m1', m2'} D_{m1, m1'} * D_{m2, m2'} * C_{m1', m2', m3} = \sum_{m3'} C_{m1, m2, m3'} * D_{m3', m3}
    \sum_{m1, m2, m1', m2', m3'} D_{m1, m1'} * D_{m2, m2'} * C_{m1', m2', m3'} * C_{m1, m2, m3}  = D_{m1, m2}
    """
    # pick random angles
    N = 1
    
    n1 = 2*l1 + 1
    n2 = 2*l2 + 1
    n3 = 2*l3 + 1
    for i in range(N):
        g = np.random.rand(3) * 2 * np.pi
        D1 = wigner_d_num(g, l1)
        D2 = wigner_d_num(g, l2)
        D3 = wigner_d_num(g, l3)
        
        lhs = 42*np.ones(shape=(n1,n2,n3))
        rhs = 42*np.ones(shape=(n1,n2,n3))

        for m1 in range(n1):
            for m2 in range(n2):
                lhs[m1,m2,:] = D1[m1,:] * D2[m2,:]
            
        #for m3 in range(n2):
        #    rhs[:,:,m3] = D3[m3,:]

    return None
        #print("lhs:", lhs)
        #print("rhs:", rhs)

clebsch_gordan_num(l1=1, l2=1, l3=0)
    
#g = [0., 0, 0]
#print(rot_mat(g))
#print(wigner_d_num(g=g, l=2))

ValueError: could not broadcast input array from shape (3) into shape (1)

In [ ]:
import lie_learn

In [ ]:
r.as_dcm

In [ ]:
alpha, beta, gamma = np.random.rand(3) * 2 * np.pi

In [ ]:
alpha